In [ ]:
# Here we are going to collect all the d

In [608]:
import pandas as pd

In [609]:
## Functions for getting consistent names of the four classes
def rep1 (text):
    d = text.replace('indication_treatment', 'treatment')
    e = d.replace('indication_symptomatic_relief', 'relief')
    return e

def rep2 (text):
    d = text.replace('INDICATIONS AND USAGE', '')
    e = d.replace('\n', '')
    return e


def rep3 (text):
    a = text.replace('indication_treatment','treatment')
    b = a.replace('indication_symptomatic_relief','relief')
    c = b.replace('i_dont_know', 'IDK')
    d = c.replace('unrelated', 'IDK')
    e = d.replace('Indication: Symptomatic relief', 'relief')
    f = e.replace('Indication: treatment', 'treatment')
    g = f.replace('I dont know', 'IDK')
    h = g.replace('Effect', 'effect')
    m = h.replace('Contraindication', 'contraindication')
    return m


def rep4 (text):
    a = text.replace('Indication: Treatment','treatment')
    b = a.replace('Indication: Symptomatic Relief','relief')
    c = b.replace('No consensus', 'IDK')
    d = c.replace('IDK', 'IDK')
    e = d.replace('Effect', 'effect')
    f = e.replace('Contraindication', 'contraindication')
    return f

# get everything in lowercase
def lower (x):
    return x.lower()

# functions for mapping DOID with disease names
def map_doid (code):
    ind = doid['DOID'].index[ doid['DOID']== code].tolist()
    name = doid['disease'][ind[0]]
    return name

# functions for mapping DBID with drug names
def map_db (code):
    ind = db['DBID'].index[ db['DBID']== code].tolist()
    name = db['drug'][ind[0]]
    return name

In [610]:
# The datasets of the DOID and DBID codes 

doid = pd.read_csv('DOID - Sheet1.csv')
doid = doid.rename(columns={"Disease Name": "disease",})

db = pd.read_csv('DB - Sheet1.csv')
db = db.rename(columns={"Drug Name": "drug", "DrugBank ID":"DBID"})

## SET ONE (with label id)

In [626]:
crowd1 = pd.read_csv('new_data.csv')
crowd1 = crowd1[['text' ,'disease_name', 'do_id', 'drug_name','drugbank_id', 'relation_type', 'label_id']]
crowd1 = crowd1.rename(columns={"disease_name": "disease", "do_id": "DOID", "drugbank_id": "DBID", "relation_type": "relation", "drug_name": "drug"})
crowd1['relation'] = crowd1['relation'].apply(rep1)
crowd1['disease'] = crowd1['disease'].apply(lower)

# delete the empty rows 
crowd1 = crowd1.dropna(subset=['DOID', 'DBID']).reset_index()
crowd1 = crowd1.drop(crowd1.index[1213]).reset_index(drop=True)

# keep the columns that we need
crowd1 = crowd1[['text' ,'disease', 'DOID', 'drug','DBID', 'relation', 'label_id']]

# map the ids
crowd1['disease'] = crowd1['DOID'].apply(map_doid)
crowd1['drug'] = crowd1['DBID'].apply(map_db)

print('lenght of first set:', len(crowd1))
crowd1.sample()

lenght of first set: 1248


,text,disease,DOID,drug,DBID,relation,label_id
373,Dantrolene sodium capsules are indicated in co...,multiple sclerosis,DOID_2377,Dantrolene,DB01219,relief,cbece5e5-72b3-433f-84ae-03013ce93454.xml


## SET TWO (with same label id)

In [627]:
crowd2 = pd.read_csv('collect_data.csv')
crowd2 = crowd2.dropna(subset=['do_id']).reset_index()

# Delete the empty rows
crowd2 = crowd2.drop(crowd2.index[1540]).reset_index(drop=True)
crowd2 = crowd2.drop(crowd2.index[2039]).reset_index(drop=True)

# keep the columns that we need and change names for consistency 
crowd2 = crowd2[['text' ,'disease_name', 'do_id', 'drugbank_id', 'relation_type', 'label_id']]
crowd2 = crowd2.rename(columns={"disease_name": "disease", "do_id": "DOID", "drugbank_id": "DBID", "relation_type": "relation" })

# clean the columns
crowd2['text'] = crowd2['text'].apply(rep2)
crowd2['relation'] = crowd2['relation'].apply(rep3)
crowd2['disease'] = crowd2['disease'].apply(lower)

data2 = {
'text': crowd2['text'],
'disease': crowd2['disease'],
'DOID': crowd2['DOID'],
'drug': 'drug',
'DBID': crowd2['DBID'],
'relation': crowd2['relation'],
'label_id': crowd2['label_id']}

# map the ids
df2 = pd.DataFrame(data2)
df2['disease'] = df2['DOID'].apply(map_doid)
df2['drug'] = df2['DBID'].apply(map_db)

print('lenght of second set:', len(df2))
df2.sample()

lenght of second set: 2141


,text,disease,DOID,drug,DBID,relation,label_id
306,RETAVASE is indicated for use in acute ST-elev...,myocardial infarction,DOID_5844,Reteplase,DB00015,treatment,966dcb73-29d5-4119-ba97-3298e384b83a.xml


## SET THREE (with label id)

In [628]:
crowd3 = pd.read_csv('crowd.csv')
crowd3 = crowd3[['context' , 'do_id', 'drug_id', 'worker_consensus', 'label_id']]

# Delete the empty rows
crowd3 = crowd3.dropna(subset=['do_id']).reset_index()
crowd3 = crowd3.drop(crowd3.index[813]).reset_index(drop=True)
crowd3 = crowd3.rename(columns={ "do_id": "DOID", "drug_id": "DBID", "worker_consensus": "relation", "context": "text" })
crowd3['relation'] = crowd3['relation'].apply(rep4)

data3 = {
'text': crowd3['text'],
'disease': 'disease',
'DOID': crowd3['DOID'],
'drug': 'drug',
'DBID': crowd3['DBID'],
'relation': crowd3['relation'],
'label_id': crowd3['label_id']}

df3 = pd.DataFrame(data3)

# map the ids
df3['disease'] = df3['DOID'].apply(map_doid)
df3['drug'] = df3['DBID'].apply(map_db)

print('lenght of third set:', len(df3))
df3.sample()

lenght of third set: 1790


,text,disease,DOID,drug,DBID,relation,label_id
597,Allergenic extracts should not be used if the ...,allergic hypersensitivity disease,DOID_1205,Artemisia douglasiana pollen,DB10761,treatment,../DailyMedExtracter/prescription/temp_xml/35b...


## SET FOUR (no label id)

In [629]:
crowd4 = pd.read_csv('crowd2.csv')
crowd4 = crowd4[['context' , 'do_id', 'drug_id', 'worker_consensus']]
crowd4 = crowd4.rename(columns={ "do_id": "DOID", "drug_id": "DBID", "worker_consensus": "relation", "context": "text" })
crowd4['relation'] = crowd4['relation'].apply(rep4)

data4 = {
'text': crowd4['text'],
'disease': 'disease',
'DOID': crowd4['DOID'],
'drug': 'drug',
'DBID': crowd4['DBID'],
'relation': crowd4['relation']}

df4 = pd.DataFrame(data4)

df4['disease'] = df4['DOID'].apply(map_doid)
df4['drug'] = df4['DBID'].apply(map_db)

print('lenght of third set:', len(df4))
df4.sample()

lenght of third set: 1354


,text,disease,DOID,drug,DBID,relation
44,"Glaucoma; obstructive uropathy (e.g., bladder ...",colitis,DOID_0060180,Methscopolamine,DB11315,effect


## Merge all the sets

In [634]:
sets = [crowd1, df2, df3, df4]
merged = pd.concat(sets, ignore_index=True,  sort=True)
print('lenght of all:', len(merged))

# Delete the dublicates rows
df = merged.drop_duplicates(subset=['text', 'DOID', 'DBID','relation' ], keep="first",).reset_index()
cleaned = df.drop(df.columns[0], axis = 1)
print('lenght of final set:', len(cleaned))
cleaned.sample()

lenght of all: 6533
lenght of final set: 4046


,DBID,DOID,disease,drug,label_id,relation,text
564,DB00995,DOID_7148,rheumatoid arthritis,Auranofin,6008addc-7f8e-45bb-8aec-54dd54a788ac.xml,treatment,RIDAURA (auranofin) is indicated in the manage...


In [637]:
# Save the set in a new data frame 

cleaned.to_csv('crowd_all.csv')